<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

Coupled Trigger Test

This example creates 3 waves of different types, download the waves once for each DDR (depending on the number of channels in the device). Creates a task table. Download it for each channel and plays the task table according to a trigger (TRG1).triggered. And finally print registers status.

In [17]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [18]:
import os
import gc
import sys
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
#from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst

In [19]:
# Connect to instrument

# Please choose appropriate address:
inst_addr = 'TCPIP::192.168.0.101::5025::SOCKET'

inst = TEVisaInst(inst_addr)

resp = inst.send_scpi_query("*IDN?")
print('connected to: ' + resp)

connected to: Tabor Electronics,P25812B,000000220726,1.118.0


In [20]:
# Set the default paranoia level. 
# It controls the default behavior of: inst.send_scpi_cmd(scpi_str).
#  - paranoia-level = 0: send scpi_str as is.
#  - paranoia-level = 1: append '*OPC?' to scpi_str, and send it as query 
#  - paranoia-level = 2: append ':SYST:ERR?' to scpi_str, send it as query, and print warnning in case of error.

inst.default_paranoia_level = 2 # paranoia-level = 2 (good for debugging)

# Reset the instrument
inst.send_scpi_cmd('*CLS; *RST')
resp = inst.send_scpi_query(":SYST:ERR?")
print(resp)

0, no error


In [21]:
# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get model dependant parameters:

if model_name.startswith('P948'):
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    channels_per_dac = 2
elif model_name.startswith('P908'):
    bpp = 1
    max_dac = 255
    wpt_type = np.uint8
    channels_per_dac = 1
else:
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    channels_per_dac = 2
    
half_dac = max_dac / 2.0

# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = (int(resp) // 64) * 64
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))

max_seglen = arbmem_capacity // bpp
print ('Max segment length: {0:,}'.format(max_seglen))

Model: P25812B 
Number of channels: 12
Max segment number: 65536
Available memory per DDR: 4,286,578,688 wave-bytes
Max segment length: 2,143,289,344


In [22]:
# Build 3 waveforms

seglen = 4096
cyclelen = seglen
ncycles = seglen / cyclelen
waves = [ None for _ in range(3)]

# sin wave:
x = np.linspace(
    start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
y = (np.sin(x) + 1.0) * half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
waves[0] = y.astype(wpt_type)

# triangle wave:
x = np.linspace(
    start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
y = np.sin(x)
y = np.arcsin(y)* 2 * half_dac / np.pi + half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
waves[1] = y.astype(wpt_type)

# square wave
x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
y = np.fmod(x, cyclelen)
y = (y <= cyclelen / 2) * max_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
waves[2] = y.astype(wpt_type)

In [23]:
# download 3 waveforms to each DDR

for ichan in range(num_channels):
    if ichan % channels_per_dac == 0:
        channb = ichan + 1
        # Select channel
        cmd = ':INST:CHAN {0}'.format(channb)
        inst.send_scpi_cmd(cmd)
        for iseg in range(3):
            segnum = iseg + 1
            print('Downloading segment {0} of channel {1}'.format(segnum, channb))
            # Define segment
            cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
            inst.send_scpi_cmd(cmd)
            
            # Select the segment
            cmd = ':TRAC:SEL {0}'.format(segnum)
            inst.send_scpi_cmd(cmd)
            
            # Send the binary-data:
            inst.write_binary_data(':TRAC:DATA', waves[iseg])
            
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error


In [24]:
# Play the first segment in each channel

for ichan in range(num_channels):
    channb = ichan + 1
    # Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd)
    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(1)
    inst.send_scpi_cmd(cmd)

    # Turn on the output of the selected channel:
    inst.send_scpi_cmd(':OUTP ON')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error


In [25]:
inst.send_scpi_cmd(':INST:CHAN 1')

inst.send_scpi_cmd(':XINS:SYNC:STATE ON')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error


In [26]:
# Set the thrshhold level of trigger1 of the master module

inst.send_scpi_cmd(':INST:CHAN 1')

inst.send_scpi_cmd('TRIG:SEL TRG1')

inst.send_scpi_cmd('TRIG:STAT ON')

inst.send_scpi_cmd(':TRIG:LEV 0.2')


# Turn on trigger-coupling 
# This means that trigger 1 of the master-module goes to all modules.
# Temporarily, the enabling-signal should be INTernal instead of TRG1.
inst.send_scpi_cmd(':TRIG:COUPLE ON')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error


In [27]:
# Define task-table of 3 tasks in each channel.
# The first task shall wait for trigger1.
# In order to 

tasklen = 3

# Compose the task-table rows:
cmd = ':TASK:COMP:LENG {0}'.format(tasklen)
inst.send_scpi_cmd(cmd)

for itask in range(tasklen):
    tasknb = itask + 1
    segnb = itask + 1
    nloops = 2 ** tasknb
    
    cmd = ':TASK:COMP:SEL {0}'.format(tasknb)
    inst.send_scpi_cmd(cmd)
    
    inst.send_scpi_cmd(':TASK:COMP:TYPE SING')
    
    cmd = ':TASK:COMP:SEGM {0}'.format(segnb)
    inst.send_scpi_cmd(cmd)
    
    cmd = ':TASK:COMP:LOOP {0}'.format(nloops)
    inst.send_scpi_cmd(cmd)
    
    if 1 == tasknb:
        # Temporarily put INT instead of TRG1 
        # in case of :TRIG:COUPLE ON
        cmd = ':TASK:COMP:ENAB INT' 
        inst.send_scpi_cmd(cmd)
    else:
        cmd = ':TASK:COMP:ENAB NONE'
        inst.send_scpi_cmd(cmd)
        
    if tasklen == tasknb:
        cmd = ':TASK:COMP:NEXT1 1'
        inst.send_scpi_cmd(cmd)
    else:
        cmd = ':TASK:COMP:NEXT1 {0}'.format(tasknb + 1)
        inst.send_scpi_cmd(cmd)

# write the task table rows to the task-table of each channel
for ichan in range(num_channels):
    channb = ichan + 1
    # Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd)
    # Write the task-table
    inst.send_scpi_cmd(':TASK:COMP:WRIT')
    
    # Set Task-Mode
    inst.send_scpi_cmd(':FUNC:MODE TASK')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error


In [28]:
# Set sampling clock rate
inst.send_scpi_cmd(':FREQ:RAST 2.5e9')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error


In [29]:
base_addr = {
    'global': 0x800000,
    'gpio': 0x801000,
    'chan1': 0x803000,
    'chan2': 0x804000,
    'chan3': 0x008000,
    'chan4': 0x009000,
    'marker': 0x400000,
    'trig1':  0x815000,
    'trig2':  0x816000,
    'trig3':  0x00d000,
    'trig4':  0x00e000
}

def write_register(ip_name, reg_addr, reg_val):
    global inst
    global base_addr
    
    addr = base_addr[ip_name] + reg_addr
    inst.send_scpi_cmd('SHELL:REG {0}, {1}'.format(addr, reg_val))
    
def read_register(ip_name, reg_addr):
    global inst
    global base_addr
    
    addr = base_addr[ip_name] + reg_addr
    reg_val_str = inst.send_scpi_query('SHELL:REG? {0}'.format(addr))
    
    print('{0} reg[0x{1:02x}] = {2}'.format(ip_name, reg_addr, reg_val_str))
    
    if reg_val_str.startswith('0x') or reg_val_str.startswith('0X'):
        reg_val = int(reg_val_str[2:], 16)
    else:
        reg_val = np.uint32(reg_val_str)
    return reg_val

In [30]:
val = read_register('global', 0x18)
write_register('global', 0x18, val)

global reg[0x18] = 0x00000021


In [31]:
# Print Registers

import tempfile
import webbrowser


channb = 1

# query the registers info (as html)
cmd = ':INST:CHAN {0}; :SYST:INF:REG?'.format(channb)
html_str = inst.send_scpi_query(cmd, max_resp_len=200000)

#print(html_str)
with tempfile.NamedTemporaryFile('w', delete=False, suffix='.html') as f:
    url = 'file://' + f.name
    f.write(html_str)
    
webbrowser.open(url)

True

In [32]:
inst.close_instrument()